### Feature engineering using orders and vendors information ###

During our initial analysis we decided to remove many columns from train_full. Here we will be adding in columns based on vendor and order information that matches with the customer and vendor in question. 

Below is a list of features we want to add: 
- Has ordered here before: binary (orders.csv) 
- Grand total: Average across customer’s order from vendor. (orders.csv) 
- Is_favorite: Set missing equal to 0, 1 otherwise. (orders.csv) 
- Distance between customer location and vendor location (train_full) 
- Number of times customer has ordered from this vendor tag (orders.csv) 
- One-hot encoding of vendor tag (train_full/vendors.csv) 
- Is_rated and vendor_rating (orders.csv) 

In [ ]:
import pandas as pd
import numpy as np

# Read in CSV
orders = pd.read_csv('../data/orders.csv')
vendors = pd.read_csv('../data/vendors.csv')

# Gathering the list of columns we removed from train_full.csv
columns = ['is_open', 'status_y', 'device_type', 'verified_y', 'commission', 'is_akeed_delivering', 'language', 'open_close_flags', 'one_click_vendor', 'country_id', 'city_id', 'display_orders', 'gender', 'location_type', 'OpeningTime', 'OpeningTime2', 'sunday_from_time1', 'sunday_to_time1', 'sunday_from_time2', 'sunday_to_time2', 'monday_from_time1', 'monday_to_time1', 'monday_from_time2', 'monday_to_time2', 'tuesday_from_time1', 'tuesday_to_time1', 'tuesday_from_time2', 'tuesday_to_time2', 'wednesday_from_time1', 'wednesday_to_time1', 'wednesday_from_time2', 'wednesday_to_time2', 'thursday_from_time1', 'thursday_to_time1', 'thursday_from_time2', 'thursday_to_time2', 'friday_from_time1', 'friday_to_time1', 'friday_from_time2', 'friday_to_time2', 'saturday_from_time1', 'saturday_to_time1', 'saturday_from_time2', 'saturday_to_time2', 'primary_tags', 'status_x', 'verified_x', 'created_at_x', 'updated_at_x', 'authentication_id', 'vendor_category_en', 'rank', 'created_at_y', 'updated_at_y']

# Drop columns from the dataframes if they were removed from train_full.csv
for col in columns:
    if col in orders.columns:
        orders = orders.drop(col, axis=1)
    if col in vendors.columns:
        vendors = vendors.drop(col, axis=1)
    
        
# Drop rows in orders where the location_number != 0
orders = orders[orders['LOCATION_NUMBER'] == 0]

print("Cleaned up dataframes columns")
print(orders.columns)
print(vendors.columns)

Index(['akeed_order_id', 'customer_id', 'item_count', 'grand_total',
       'payment_mode', 'promo_code', 'vendor_discount_amount',
       'promo_code_discount_percentage', 'is_favorite', 'is_rated',
       'vendor_rating', 'driver_rating', 'deliverydistance', 'preparationtime',
       'delivery_time', 'order_accepted_time', 'driver_accepted_time',
       'ready_for_pickup_time', 'picked_up_time', 'delivered_time',
       'delivery_date', 'vendor_id', 'created_at', 'LOCATION_NUMBER',
       'LOCATION_TYPE', 'CID X LOC_NUM X VENDOR'],
      dtype='object')
Index(['id', 'latitude', 'longitude', 'vendor_category_id', 'delivery_charge',
       'serving_distance', 'prepration_time', 'discount_percentage', 'status',
       'verified', 'vendor_rating', 'vendor_tag', 'vendor_tag_name',
       'created_at', 'updated_at'],
      dtype='object')


/var/folders/9t/xpbsmtsx0xzbs7_6z34b_5yr0000gn/T/ipykernel_92016/1641897437.py:5: DtypeWarning: Columns (15,16,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  orders = pd.read_csv('../data/orders.csv')


In [ ]:
# print(orders['LOCATION_NUMBER'].value_counts())
print(orders['is_rated'].value_counts())
print(orders['vendor_rating'].value_counts())